## Содержание:
* [1. Отправка списка страниц на индексирование в Google Indexing API](#1)
* [-- Настройки](#2)
* [-- Загрузка файла xlsx с url для индексирования](#3)
* [-- Отправка на индексирование](#4)
<br>
<br>
* [1. Отправка списка страниц в Google Indexing API c обходом ограничения по квоте](#5)
* [-- Настройки](#6)
* [-- Загрузка файла xlsx с url для индексирования](#7)
* [-- Отправка на индексирование](#8)

## Пакет для работы, если не был установлен ранее

In [ ]:
#pip install google-api-python-client

# Отправка списка страниц на индексирование в Google Indexing API<a class="anchor" id="1"></a>

In [1]:
import json
import time
import requests

import httplib2
from oauth2client.service_account import ServiceAccountCredentials

import pandas as pd

## Настройки<a class="anchor" id="2"></a>

### Ресурсы для отправки

In [2]:
SCOPES = ["https://www.googleapis.com/auth/indexing"]
ENDPOINT = "https://indexing.googleapis.com/v3/urlNotifications:publish"

**параметры проекта**

In [3]:
key = 'davita-5016199-74685386f70z1b' # укажите название скачанного json файла без расширения

In [4]:
project_name = 'davita'

## Загрузка URL для индексирования<a class="anchor" id="3"></a>

Файл в формате xlsx с названием поля "urls":

In [5]:
table_with_urls_for_recrawl = pd.read_excel(f'{project_name}.xlsx')

## Отправка на индексирование <a class="anchor" id="4"></a>

Функция принимает множество (set) url и ключ,<br>
отправленные адреса добавляет в отдельный set,<br>
пишет ответ и количество отправленных страниц,<br>
возвращает set отправленных страниц.

In [6]:
def send_pages_to_google_for_recrawl(data, key):

    json_key_file = f"{key}.json"
    credentials = ServiceAccountCredentials.from_json_keyfile_name(json_key_file, 
                                                                   scopes=SCOPES)
    http = credentials.authorize(httplib2.Http())

    sent_urls_for_recrawl_set = set()
    for url in data:
        
        urls = {'url': '{}'.format(url),
                'type': 'URL_UPDATED'}

        response, content = http.request(ENDPOINT, method="POST", 
                                        body=json.dumps(urls))

        time.sleep(1)

        if response['status'] != '200':
            break

        sent_urls_for_recrawl_set.add(url)

    sent_urls_set_len = len(sent_urls_for_recrawl_set)

    print(response['status'])
    print(content.decode())

    print('На переобход отправлено:', sent_urls_set_len)
    
    return sent_urls_for_recrawl_set

### Удаление из списка и экспорт

Функция принимает set с со всеми адресами из загруженной таблицы и set отправленных url,<br>
Из загруженной таблицы удаляются url, которые были отправлены. Это нужно для того, чтобы в следующий раз, когда обновится квота, эти же страницы не оправлялись на переобход.

**Экспорт**

In [7]:
def export_data_to_excel(data):
    project_report = pd.ExcelWriter(f'{project_name}.xlsx', engine='xlsxwriter')
    data.to_excel(project_report, index=False)
    project_report.save()

**Удаление и экспорт**

In [8]:
def delete_sent_urls_and_export_new_table(main_urls_set, sent_urls_for_recrawl_set):
    
    main_urls_set_without_sent_urls = main_urls_set - sent_urls_for_recrawl_set
        
    rest_urls_list = list(main_urls_set_without_sent_urls)
    
    data = pd.DataFrame({'urls': rest_urls_list})
        
    export_data_to_excel(data)

## Главная функция

Принимает датафрейм, который был загружен в начале,<br>
преобразует датафрейм в set,<br>
отправляет на переобход,<br>
удаляет из общего списка отправленные url и экспортирует датафрейм.

In [13]:
def main_send_pages_to_google_for_recrawl(data):
    
    main_urls_set = set(data['urls'].to_list())
    
    sent_urls_for_recrawl_set = send_pages_to_google_for_recrawl(main_urls_set, key)
    
    delete_sent_urls_and_export_new_table(main_urls_set, sent_urls_for_recrawl_set)

## Запуск

In [14]:
main_send_pages_to_google_for_recrawl(table_with_urls_for_recrawl)

429
{
  "error": {
    "code": 429,
    "message": "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:955122624583'.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.ErrorInfo",
        "reason": "RATE_LIMIT_EXCEEDED",
        "domain": "googleapis.com",
        "metadata": {
          "quota_metric": "indexing.googleapis.com/v3_publish_requests",
          "quota_limit": "DefaultPublishRequestsPerDayPerProject",
          "quota_location": "global",
          "consumer": "projects/955122624583",
          "service": "indexing.googleapis.com",
          "quota_limit_value": "200"
        }
      },
      {
        "@type": "type.googleapis.com/google.rpc.Help",
        "links": [
          {
            "description": "Request a higher quota limit.",
            "url": "https://cloud.google.com/docs/quota#requesting_highe

# Отправка списка страниц в Google Indexing API больше предоставленной квоты<a class="anchor" id="5"></a>

In [ ]:
import json
import time
import requests
import traceback

import httplib2
from oauth2client.service_account import ServiceAccountCredentials

import pandas as pd

## Настройки<a class="anchor" id="6"></a>

### Ресурсы для отправки

In [ ]:
SCOPES = ["https://www.googleapis.com/auth/indexing"]
ENDPOINT = "https://indexing.googleapis.com/v3/urlNotifications:publish"

**параметры проекта**

In [ ]:
key_path = '{}/{}.json'

In [ ]:
project_name = 'koleso'

## Загрузка URL для индексирования<a class="anchor" id="7"></a>


In [ ]:
table_with_urls_for_recrawl = pd.read_csv(f'{project_name}.csv')

## Обработка ошибок

In [15]:
def error_report():
    print('Ошибка')
    traceback.print_exc()

## Отправка на индексирование <a class="anchor" id="8"></a>

Функция принимает множество (set) url, ключ и email,<br>
отправленные адреса добавляет в отдельный set,<br>
пишет ответ и количество отправленных страниц,<br>
возвращает set отправленных страниц.

In [ ]:
def send_pages_to_google_for_recrawl(data, email, key):

    json_key_file = key_path.format(email, key)
    
    credentials = ServiceAccountCredentials.from_json_keyfile_name(json_key_file, 
                                                                   scopes=SCOPES)
    http = credentials.authorize(httplib2.Http())

    sent_urls_for_recrawl_set = set()
    for url in data:

        urls = {'url': '{}'.format(url),
                'type': 'URL_UPDATED'}

        response, content = http.request(ENDPOINT, method="POST", 
                                        body=json.dumps(urls))

        time.sleep(0.5)

        if response['status'] != '200':
            break

        
        sent_urls_for_recrawl_set.add(url)
        
    #print(response['status'])
    for keys,values in json.loads(content.decode())['error'].items():
        print(values,'\n')

    sent_urls_set_len = len(sent_urls_for_recrawl_set)

   
    print('На переобход отправлено:', sent_urls_set_len,'\n')

    return sent_urls_for_recrawl_set

### Удаление из списка и экспорт

Функция принимает set с со всеми адресами из загруженной таблицы и set отправленных url,<br>
Из загруженной таблицы удаляются url, которые были отправлены. Это нужно для того, чтобы в следующий раз, когда обновится квота, эти же страницы не оправлялись на переобход.

In [16]:
def delete_sent_urls_and_export_new_csv_table(main_urls_set, sent_urls_for_recrawl_set):
    
    print('Удаление отправленных URL','\n')
    
    main_urls_set_without_sent_urls = main_urls_set - sent_urls_for_recrawl_set
    
    print('Длина общего списка после удаления отправленных URL:', len(main_urls_set_without_sent_urls),'\n')
    
    rest_urls_list = list(main_urls_set_without_sent_urls)
    
    data = pd.DataFrame({'urls': rest_urls_list})
    
    print('Экспорт CSV','\n','\n')
    
    data.to_csv(f'{project_name}.csv', index=False)

## Главная функция

Открывает таблицу «settings_indexing_api», из которой берется email и название ключа,<br>
открывается csv-файл с адресами на отправку, конвертируется в set,<br>
затем отправляется по доступной квоте,<br>
отправленные адреса удаляются из общего списка, экспортируется csv-файл.

In [ ]:
def main_send_pages_to_google_for_recrawl():
    
    settings = pd.read_excel('google_indexing_settings.xlsx').to_dict('records')
    
    total_sent_urls_counter = 0
    
    for row in settings:
        try:
            email, service_email, key = row['email'], row['service_account_email'], row['key']

            print(email)
            print(service_email,'\n')

            data = pd.read_csv(f'{project_name}.csv')
            
            print('Длина общего списка URL:', len(data))
            print('Конвертация в set')
            
            main_urls_set = set(data['urls'].to_list())
            
            print('Длина списка URL после конвертации в set:', len(main_urls_set),'\n')

            sent_urls_for_recrawl_set = send_pages_to_google_for_recrawl(main_urls_set, email, key)

            delete_sent_urls_and_export_new_csv_table(main_urls_set, sent_urls_for_recrawl_set)

            total_sent_urls_counter += len(sent_urls_for_recrawl_set)
            
            print('Ожидание 20 сек.','\n','\n')
            time.sleep(20)
            
        except:
            error_report()
            
    print('На переобход отправлено:', total_sent_urls_counter) 

## Запуск

In [ ]:
main_send_pages_to_google_for_recrawl()